Read existing OSM Schutzgebiete from geojson export
Read (updated) DAV Shapefile

compare to find new or changed polygons and create an epsg:4326 shape file only with changed/new polygons for import in JOSM

Note: due to some epsg:4326 <-> epsg:31468 reprojection issue the geojson slightly (~ 2 meters) deviates from the DAV shapes
-> fuzzy matching with intersection over union approach is taken

Deleted polygons are not yet detected

In [1]:
import fiona
import pyproj
from shapely.geometry import shape, mapping
from shapely.ops import transform
from shapely.geometry import Polygon
from rdp import rdp

import copy

# for tests
# origShapeFile = 'E:/OSM/Schutzgebiete/Rauhkopf.geojson' # epsg:4326
# newShapeFile = 'E:/OSM/Schutzgebiete/Rauhkopf/Rauhkopf.shp' # epsg:31468

# real files
origShapeFile = 'E:/OSM/Schutzgebiete/Schongebiete.geojson' # epsg:4326
newShapeFile = 'E:/OSM/Schutzgebiete/200126_Schutzgebiete_By-Karten/Schutzgebiete_BY-Karten-epsg4326.shp' # epsg:4326
# newShapeFile = 'E:/OSM/Schutzgebiete/200126_Schutzgebiete_By-Karten/Schutzgebiete_BY-Karten.shp' # epsg:31468
# newShapeFile = 'E:/OSM/Schutzgebiete/Schongebiete-Alt/Schongebiete.shp' # epsg:31468

shapesUpdateFile = 'E:/OSM/Schutzgebiete/New/new-shapes.shp'
shapesIdenticalFile = 'E:/OSM/Schutzgebiete/New/same-shapes.shp'

# project = pyproj.Transformer.from_proj(
#     pyproj.Proj(init='epsg:4326'),
#     pyproj.Proj(init='epsg:31468'))

oldFeatures = []
with fiona.open(origShapeFile) as input:
    oldCrs = input.crs
    for feat in input:
        if feat['geometry'] != None and len(feat['geometry']['coordinates'][0]) > 2:
            # poly = shape(feat['geometry'])
            # poly = transform(project.transform, poly)
            # feat['geometry'] = mapping(poly)
            oldFeatures.append(feat)
size = len(oldFeatures)
print(f"OSM-Gebiete in {origShapeFile}: {size} / {oldCrs['init']}")

OSM-Gebiete in E:/OSM/Schutzgebiete/Schongebiete.geojson: 651 / epsg:4326


In [2]:
# import matplotlib.pyplot as plt
# poly = shape(oldFeatures[300]['geometry'])
# x,y = poly.exterior.xy
# plt.plot(x,y)

In [3]:
# read and 3D to 2D convert and reproject DAV shapefile

newFeatures = []
with fiona.open(newShapeFile) as input:
    schema = input.schema
    crs = input.crs
    # if crs['init'] != "epsg:31468":
    #     print(f"Bad CRS {crs['init']} in {newShapeFile}")
    #     exit

    driver = input.driver
    for feat in input:
        if 'Kategorie' in feat['properties'] and feat['properties']['Kategorie'] != None and feat['properties']['Kategorie'] == "FFH-Gebiet (A)":
            print("Ignoring FFH-Gebiet (A)")
            continue
        if feat['geometry'] != None:
            if len(feat['geometry']['coordinates']) > 1:
                # multipolygons - code to be improved...
                for pfeat in feat['geometry']['coordinates']:
                    try: # some are len(1) lists, some aren't ?!
                        poly = Polygon(pfeat[0])
                    except:
                        poly = Polygon(pfeat)
                    poly = transform(lambda x, y, z=None: (x, y), poly)
                    feat2 = copy.deepcopy(feat)
                    feat2['geometry'] = mapping(poly)
                    newFeatures.append(feat2)
                continue
            if len(feat['geometry']['coordinates'][0]) < 3:
                print("Skipping 2-point line")
                continue
            # transform 3D to 2D
            poly = shape(feat['geometry'])
            poly = transform(lambda x, y, z=None: (x, y), poly)
            feat['geometry'] = mapping(poly)
            newFeatures.append(feat)

size = len(newFeatures);          
print(f"Gebiete in {newShapeFile}: {size}")

Ignoring FFH-Gebiet (A)
Ignoring FFH-Gebiet (A)
Ignoring FFH-Gebiet (A)
Ignoring FFH-Gebiet (A)
Ignoring FFH-Gebiet (A)
Gebiete in E:/OSM/Schutzgebiete/200126_Schutzgebiete_By-Karten/Schutzgebiete_BY-Karten-epsg4326.shp: 410


In [4]:
# https://www.reddit.com/r/gis/comments/mcw0y0/comparing_two_linestrings_with_shapely/

# from collections import OrderedDict
# dummyProps = OrderedDict([('Id', None),('Name', ''),('Regelung', '')])

newCount = 0 
foundCount = 0
newFeaturesOut = []
sameFeatures = []

for newFeature in newFeatures:
    # apply a buffer to avoid TopologyException: Input geom 1 is invalid: Self-intersection at or near point
    # https://www.programmersought.com/article/69515213493/
    newGeom = Polygon(shape(newFeature['geometry']).exterior)
    newGeomB = newGeom.buffer(0.0001)
    found = False
    for oldFeature in oldFeatures:
        try:
            oldGeom = Polygon(shape(oldFeature['geometry']))
            oldGeomB = oldGeom.buffer(0.0001)
        except Exception as ex:
            # FIXME: 'MultiPolygon' object has no attribute 'exterior' - why ?!?! - MultiPolygons get removed above?!
            # print(ex)
            # print(oldFeature['geometry'])
            continue
        try:
            # this doesn't work on epsg:31468 coordinates
            # if oldGeom.almost_equals(newGeom, decimal=0):
            #     iou = 1
            # else:
            #     iou = 0
            # https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
            iou = newGeomB.intersection(oldGeomB).area / newGeomB.union(oldGeomB).area
        except Exception as ex:
            print(ex)
            print(newGeom)
            print(oldGeom)
        if iou > 0.995: # baseline 0.985
            found = True
            foundCount +=1
            sameFeatures.append(newFeature)
    if not found:
        newCount += 1
        newFeaturesOut.append(newFeature)

with fiona.open(shapesUpdateFile, 'w', crs={'init':'epsg:4326'}, driver='ESRI Shapefile', schema=schema) as out:
    for f in newFeaturesOut:
        out.write(f)

with fiona.open(shapesIdenticalFile, 'w', crs={'init':'epsg:4326'}, driver='ESRI Shapefile', schema=schema) as out:
    for f in sameFeatures:
        out.write(f)

print(f"Gefundene Gebiete = {foundCount}") # 368
print(f"Neue/Geänderte Gebiete = {newCount} -> in {shapesUpdateFile}") # 42

Gefundene Gebiete = 368
Neue/Geänderte Gebiete = 42 -> in E:/OSM/Schutzgebiete/New/new-shapes.shp
